In [1]:
import re
import arxiv
import autogen

default_llm_config ={
    "request_timeout": 600,
    "seed": 44,  # change the seed for different trials
    "config_list": autogen.config_list_from_json(
        "OAI_CONFIG_LIST",
        filter_dict={"model": ["gpt-4"]},
    ),
    "temperature": 0,
}
print(default_llm_config)

{'request_timeout': 600, 'seed': 44, 'config_list': [{'model': 'gpt-4', 'api_key': '5d050ffec2b94f5eb43c54c80149561e', 'api_base': 'https://futu-002-caeast-001.openai.azure.com/', 'api_type': 'azure', 'api_version': '2023-07-01-preview'}], 'temperature': 0}


In [2]:
def search_arxiv(query, max_results=10):
    query = re.sub("\s+", " ", query)
    query_parts = query.split(" ")
    query_ready = "+AND+".join(map(lambda p: f"all:{p}", query_parts))
    print(f"修改后的query: {query_ready}")
    # base_url = "http://export.arxiv.org/api/query?"
    search = arxiv.Search(query=query, 
                          max_results=max_results, 
                          sort_by=arxiv.SortCriterion.Relevance, 
                          sort_order=arxiv.SortOrder.Descending)
    output = []
    for paper in search.results():
        output.append({"title": paper.title, "url": paper.entry_id})
    return output

In [3]:
# create an AssistantAgent instance named "assistant"
llm_config = default_llm_config.copy()
llm_config["functions"] = [
    {
        "name": "search_arxiv",
        "description": "To find arxiv papers related to user's query, use the arxiv API to search for relevant papers. Queries the arxiv API and prints the titles and URLs of the top n papers.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "user's input search_query towards arxiv api.",
                },
                "max_results": {
                    "type": "integer",
                    "description": "set the top n papers to fetch. Default to 10."
                }
            },
            "required": ["query"],
        },
    }
]
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config=llm_config,
    # is_termination_msg=lambda x: True if "TERMINATE" in x.get("content") else False,
)

In [4]:
# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    # is_termination_msg=lambda x: True if "TERMINATE" in x.get("content") else False,
    max_consecutive_auto_reply=10,
    code_execution_config={
        "work_dir": "work_dir",
        "use_docker": False,
    },
    function_map={"search_arxiv": search_arxiv},
)

In [5]:
task1 = """
Find arxiv papers that show how are people studying trust calibration in AI based systems
"""
# print(search_arxiv("large language model"))
user_proxy.initiate_chat(assistant, message=task1)

user_proxy (to assistant):


Find arxiv papers that show how are people studying trust calibration in AI based systems


--------------------------------------------------------------------------------
assistant (to user_proxy):

***** Suggested function Call: search_arxiv *****
Arguments: 
{
"query": "trust calibration in AI based systems",
"max_results": 10
}
*************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION search_arxiv...
修改后的query: all:trust+AND+all:calibration+AND+all:in+AND+all:AI+AND+all:based+AND+all:systems
user_proxy (to assistant):

***** Response from calling function "search_arxiv" *****
[{'title': 'Who Should I Trust: AI or Myself? Leveraging Human and AI Correctness Likelihood to Promote Appropriate Trust in AI-Assisted Decision-Making', 'url': 'http://arxiv.org/abs/2301.05809v1'}, {'title': 'Trust Calibration and Trust Respect: A Method for Building Team Cohes